# Laboratorio No Calificado: Destilación del conocimiento
------------------------
 
Bienvenido, durante este laboratorio no calificado vas a realizar una técnica de compresión de modelos conocida como **destilación de conocimiento** en la cual un modelo `alumno` "aprende" de un modelo más complejo conocido como `maestro`. En concreto


1. Definir una clase `Distiller` con la lógica personalizada para el proceso de destilación.
2. 2. Entrenar el modelo `teacher` que es una CNN que implementa la regularización a través del abandono.
3. 3. Entrenar un modelo `student` (una versión más pequeña del profesor sin regularización) utilizando destilación de conocimiento.
4. 4. Entrenar otro modelo `alumno` desde cero sin destilación llamado `alumno_scratch`.
5. Compare los tres estudiantes.


Este cuaderno está basado en [this](https://keras.io/examples/vision/knowledge_distillation/) tutorial oficial de Keras. 

Si quieres una aproximación más teórica a este tema no dejes de consultar este paper [Hinton et al. (2015)](https://arxiv.org/abs/1503.02531). 

¡Vamos a empezar!

## Imports

In [ ]:
# For setting random seeds
import os
os.environ['PYTHONHASHSEED']=str(42)

# Libraries
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

# More random seed setup
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

## Preparar los datos

Para este laboratorio utilizarás el archivo [cats vs dogs](https://www.tensorflow.org/datasets/catalog/cats_vs_dogs) que está compuesto por muchas imágenes de gatos y perros junto a sus respectivas etiquetas. 

Comienza descargando los datos:

In [ ]:
# Define train/test splits
splits = ['train[:80%]', 'train[80%:90%]', 'train[90%:]']

# Download the dataset
(train_examples, validation_examples, test_examples), info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True, split=splits)

# Print useful information
num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

print(f"There are {num_examples} images for {num_classes} classes.")

Preprocesar los datos para el entrenamiento normalizando los valores de los píxeles, dándoles nueva forma y creando lotes de datos:

In [ ]:
# Some global variables
pixels = 224
IMAGE_SIZE = (pixels, pixels)
BATCH_SIZE = 32

# Apply resizing and pixel normalization
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label

# Create batches of data
train_batches      = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches       = test_examples.map(format_image).batch(1)

## Codificar el modelo personalizado `Distiller`

Para implementar el proceso de destilación crearemos un modelo Keras personalizado al que llamaremos `Distiller`. Para ello necesitarás sobreescribir algunos de los métodos vanilla de un `keras.Model` para incluir la lógica personalizada para la destilación de conocimiento. Necesitas sobreescribir estos métodos:
- `compile`:: Este modelo necesita algunos parámetros extra para ser compilado como las pérdidas del profesor y del alumno, el alfa y la temperatura.
- `train_step`: Controla cómo se entrena el modelo. Aquí será donde se encuentre la lógica real de destilación del conocimiento. Este método es lo que se llama cuando se hace `model.fit`.
- `test_step`:: Controla la evaluación del modelo. Este método es el que se utiliza cuando se ejecuta `model.evaluate`.

Para obtener más información sobre la personalización de los modelos echa un vistazo a la [docs oficial](https://keras.io/guides/customizing_what_happens_in_fit/).

In [ ]:
class Distiller(keras.Model):

  # Necesita los modelos del alumno y del profesor para crear una instancia de esta clase
  def __init__(self, student, teacher):
      super(Distiller, self).__init__()
      self.teacher = teacher
      self.student = student


  # Se utilizará al llamar a model.compile()
  def compile(self, optimizer, metrics, student_loss_fn,
              distillation_loss_fn, alpha, temperature):

      # Compilar utilizando el optimizador y las métricas
      super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
      
      # Añade los demás parámetros a la instancia
      self.student_loss_fn = student_loss_fn
      self.distillation_loss_fn = distillation_loss_fn
      self.alpha = alpha
      self.temperature = temperature


  # Se utilizará al llamar a model.fit()
  def train_step(self, data):
      # Data is expected to be a tuple of (features, labels)
      # Se espera que los datos sean una tupla de (características, etiquetas)
      x, y = data

      # Vanilla forward pass of the teacher
      # Note that the teacher is NOT trained
      # Tenga en cuenta que el maestro NO está entrenado
      teacher_predictions = self.teacher(x, training = False)

      # Use GradientTape to save gradients
      with tf.GradientTape() as tape:
          # Vanilla forward pass of the student
          student_predictions = self.student(x, training = True)

          # calcular la pérdida del alumno vainilla
          student_loss = self.student_loss_fn(y, student_predictions)
          
          # Calcular la pérdida de destilación
          # Debería ser la divergencia KL entre logits suavizada por un factor de temperatura
          distillation_loss = self.distillation_loss_fn(
              tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
              tf.nn.softmax(student_predictions / self.temperature, axis=1))

          # Calcula la pérdida ponderando las dos pérdidas anteriores utilizando el parámetro alfa
          loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

      # Utilizar tape.gradient para calcular gradientes para el alumno
      trainable_vars = self.student.trainable_variables
      gradients = tape.gradient(loss, trainable_vars)

      # Update student weights 
      # Note that this done ONLY for the student # Actualizar los pesos del estudiante 
      # Tenga en cuenta que esto sólo se hace para el estudiante
      self.optimizer.apply_gradients(zip(gradients, trainable_vars))

      # Update the metrics
      self.compiled_metrics.update_state(y, student_predictions)

      # Devuelve un diccionario de rendimiento
      # Verás que esto se emite durante el entrenamiento
      results = {m.name: m.result() for m in self.metrics}
      results.update({"student_loss": student_loss, "distillation_loss": distillation_loss})
      return results


  # Se utilizará al llamar a model.evaluate()
  def test_step(self, data):
      # Data is expected to be a tuple of (features, labels)
      x, y = data

      # Use student to make predictions
      # Notice that the training param is set to False
      y_prediction = self.student(x, training=False)

      # Calculate student's vanilla loss
      student_loss = self.student_loss_fn(y, y_prediction)

      # Update the metrics
      self.compiled_metrics.update_state(y, y_prediction)

      # Return a performance dictionary
      # You will see this being outputted during inference
      results = {m.name: m.result() for m in self.metrics}
      results.update({"student_loss": student_loss})
      return results


## Modelos de profesor y alumno

Para los modelos se utilizará una arquitectura CNN estándar que implemente regularización a través de algunas capas dropout (en el caso del profesor), pero podría ser cualquier modelo de Keras. 

Define las funciones `create_model` para crear modelos con la arquitectura deseada utilizando el [Modelo Secuencial] de Keras (https://keras.io/guides/sequential_model/).

Observa que `create_small_model` devuelve una versión simplificada del modelo (en cuanto a número de capas y ausencia de regularización) que devuelve `create_big_model`:

In [ ]:
# Teacher model
def create_big_model():
  tf.random.set_seed(42)
  model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.5),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(2)
  ])

  return model



# Student model
def create_small_model():
  tf.random.set_seed(42)
  model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(2)
  ])

  return model

Hay dos cosas importantes a tener en cuenta:
- La última capa no tiene una activación softmax porque los logits brutos son necesarios para la destilación del conocimiento.
- La regularización mediante capas de abandono se aplicará al profesor pero NO al alumno. Esto se debe a que el alumno debería ser capaz de aprender esta regularización a través del proceso de destilación.

Recuerde que el modelo del alumno puede considerarse como una versión simplificada (o comprimida) del modelo del profesor.



In [ ]:
# Create the teacher
teacher = create_big_model()

# Plot architecture
keras.utils.plot_model(teacher, rankdir="LR")

In [ ]:
# Create the student
student = create_small_model()

# Plot architecture
keras.utils.plot_model(student, rankdir="LR")

Comprueba la diferencia real en el número de parámetros entrenables (pesos y sesgos) entre ambos modelos:

In [ ]:
# Calculates number of trainable params for a given model
def num_trainable_params(model):
  return np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])


student_params = num_trainable_params(student)
teacher_params = num_trainable_params(teacher)

print(f"Teacher model has: {teacher_params} trainable params.\n")
print(f"Student model has: {student_params} trainable params.\n")
print(f"Teacher model is roughly {teacher_params//student_params} times bigger than the student model.")

### Entrenar al profesor

En la destilación de conocimiento se asume que el profesor ya ha sido entrenado, por lo que el primer paso natural es entrenar al profesor. Lo hará durante un total de 8 épocas:

In [ ]:
# Compile the teacher model
teacher.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # Notice from_logits param is set to True
    optimizer=keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

# Fit the model and save the training history (will take from 5 to 10 minutes depending on the GPU you were assigned to)
teacher_history = teacher.fit(train_batches, epochs=8, validation_data=validation_batches)

## Entrene a un alumno desde cero como referencia

Para evaluar la eficacia del proceso de destilación, entrene un modelo equivalente al alumno pero sin realizar la destilación de conocimientos. Observe que el entrenamiento se realiza durante sólo 5 épocas:

In [ ]:
# Create student_scratch model with the same characteristics as the original student
student_scratch = create_small_model()

# Compile it
student_scratch.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

# Train and evaluate student trained from scratch (will take around 3 mins with GPU enabled)
student_scratch_history = student_scratch.fit(train_batches, epochs=5, validation_data=validation_batches)

## Destilación de Conocimientos

Para realizar el proceso de destilación de conocimiento usted utilizará el modelo personalizado que codificó anteriormente. Para ello, comience creando una instancia de la clase `Distiller` y pasándole los modelos del alumno y del profesor. A continuación, compílelo con los parámetros adecuados y entrénelo.

Los dos modelos de estudiante se entrenan sólo durante 5 épocas, a diferencia del modelo de profesor, que se entrena durante 8 épocas. Esto se hace para mostrar que la destilación del conocimiento permite tiempos de entrenamiento más rápidos ya que el estudiante aprende de un modelo ya entrenado.

In [ ]:
# Create Distiller instance
distiller = Distiller(student=student, teacher=teacher)

# Compile Distiller model
distiller.compile(
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.05,
    temperature=5,
)

# Distill knowledge from teacher to student (will take around 3 mins with GPU enabled)
distiller_history = distiller.fit(train_batches, epochs=5, validation_data=validation_batches)

## Comparación de los modelos

Para comparar los modelos, puede comprobar la "precisión categórica dispersa" de cada uno de ellos en el conjunto de prueba:

In [ ]:
# Compute accuracies
student_scratch_acc = student_scratch.evaluate(test_batches, return_dict=True).get("sparse_categorical_accuracy")
distiller_acc = distiller.evaluate(test_batches, return_dict=True).get("sparse_categorical_accuracy")
teacher_acc = teacher.evaluate(test_batches, return_dict=True).get("sparse_categorical_accuracy")

# Print results
print(f"\n\nTeacher achieved a sparse_categorical_accuracy of {teacher_acc*100:.2f}%.\n")
print(f"Student with knowledge distillation achieved a sparse_categorical_accuracy of {distiller_acc*100:.2f}%.\n")
print(f"Student without knowledge distillation achieved a sparse_categorical_accuracy of {student_scratch_acc*100:.2f}%.\n")

El modelo del profesor ofrece una mayor precisión que los dos modelos de los alumnos. Esto es de esperar, ya que se entrenó durante más épocas y se utilizó una arquitectura más grande.

Observa que el estudiante sin destilación fue superado por el estudiante con destilación de conocimiento. 

Como guardaste el historial de entrenamiento de cada modelo, puedes crear un gráfico para comparar mejor los dos modelos de los estudiantes.

In [ ]:
# Get relevant metrics from a history
def get_metrics(history):
  history = history.history
  acc = history['sparse_categorical_accuracy']
  val_acc = history['val_sparse_categorical_accuracy']
  return acc, val_acc


# Plot training and evaluation metrics given a dict of histories
def plot_train_eval(history_dict):
  
  metric_dict = {}

  for k, v in history_dict.items():
    acc, val_acc= get_metrics(v)
    metric_dict[f'{k} training acc'] = acc
    metric_dict[f'{k} eval acc'] = val_acc

  acc_plot = pd.DataFrame(metric_dict)
  
  acc_plot = sns.lineplot(data=acc_plot, markers=True)
  acc_plot.set_title('training vs evaluation accuracy')
  acc_plot.set_xlabel('epoch')
  acc_plot.set_ylabel('sparse_categorical_accuracy')
  plt.show()


# Plot for comparing the two student models
plot_train_eval({
    "distilled": distiller_history,
    "student_scratch": student_scratch_history,
})

Este gráfico es muy interesante porque muestra que la versión destilada superó a la no modificada en casi todas las épocas al utilizar el conjunto de evaluación. Además, el alumno sin destilar obtiene una mayor precisión de entrenamiento, lo que indica que se está sobreajustando más que el modelo destilado. **Esto indica que el modelo destilado fue capaz de aprender de la regularización implementada por el profesor.

-----------------------------
**Ahora deberías tener una comprensión** más clara de lo que es la Destilación del Conocimiento y cómo se puede implementar utilizando Tensorflow y Keras. 

Este proceso es ampliamente utilizado para la compresión de modelos y ha demostrado funcionar muy bien. De hecho, puede que hayas oído hablar de [`DistilBert`](https://huggingface.co/transformers/model_doc/distilbert.html), que es una versión más pequeña, rápida, barata y ligera de BERT.


**Sigue así.**